In [ ]:
!pip install fastapi uvicorn faiss-cpu transformers datasets sentence-transformers torch

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from datasets import load_dataset

# Initialize FastAPI app
app = FastAPI()

# Load pre-trained sentence embedding model (MiniLM-BERT)
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Load dataset (AG News)
dataset = load_dataset("ag_news")
#news_articles = [article["text"] for article in dataset["train"][:1000]]  # Sample 1000 articles
news_articles = [article["text"] for article in dataset["train"].select(range(1000))]

# Convert news articles into embeddings
article_embeddings = model.encode(news_articles, show_progress_bar=True)

# Create FAISS index
d = article_embeddings.shape[1]  # Embedding dimensions
index = faiss.IndexFlatL2(d)
index.add(np.array(article_embeddings))

# Request model for API
class NewsRequest(BaseModel):
    article_text: str
    top_k: int = 5

# API endpoint: Recommend similar articles
@app.post("/recommend")
def recommend_articles(request: NewsRequest):
    # Convert input article to embedding
    article_embedding = model.encode([request.article_text])

    # Search for nearest neighbors
    distances, indices = index.search(np.array(article_embedding), request.top_k)

    # Retrieve recommended articles
    recommended_articles = [news_articles[i] for i in indices[0]]

    return {"input_article": request.article_text, "recommendations": recommended_articles}

# Run with: uvicorn app:app --host 0.0.0.0 --port 8000


# Request from Endpoint

**HTTP POST:** [ngrok API Endpoint](https://347b-34-83-136-115.ngrok-free.app/)

**Request Body:** `{"article_text": "Stock market crashes due to inflation", "top_k": 5}`

In [ ]:
!pip install pyngrok
!ngrok config add-authtoken <YOU_NGROK_TOKEN>

import uvicorn
from pyngrok import ngrok
import nest_asyncio

# Allow running FastAPI inside Jupyter/Colab
nest_asyncio.apply()

# Start FastAPI server in the background
port = 8000
public_url = ngrok.connect(port).public_url
print(f"FastAPI is running on {public_url}")

# Run FastAPI app
uvicorn.run(app, host="0.0.0.0", port=port)